# Baseball rules engine II

May 17, 2024

I just want to clean up what I did this morning

## 1. Make the baserunner class

In [184]:
class Baserunner:
    
    def __init__(self, name, curr_base = 0):
        
        self.name = name
        self.forced_status = False
        
        self.master_status = {'curr_obtained': curr_base, 'next_obtainable': None, 
                              'preceding_obtainable': None, 'last_obtained': curr_base, 'forced_to': None 
                             } 
        
        
    def set_master_status_unforced(self):
        curr = self.master_status['curr_obtained']
        
        self.master_status['next_obtainable'] = curr + 1
        self.master_status['preceding_obtainable'] = curr -1
    
    
    def set_master_status_forced(self):
        self.master_status['forced_to'] = self.master_status['last_obtained'] + 1
        self.master_status['last_obtained'] = None
        self.master_status['preceding_obtainable'] = None

        
    def get_base_view(self):
        base_view = {1: None, 2: None, 3: None, 4: None}
        
        for base in range(1, 5):
            for key, value in self.master_status.items():
                if value == base:
                    base_view[base] = key
        
        return base_view
                 
        
    def print_status(self):
        
        base_view = self.get_base_view()
        
        print(f"{self.name}: ", end = "")
        
        for base, status in base_view.items():
            print(f"{base}B: {status} ", end = " | ")

## 2. Instantiate 4 runners

In [166]:
def make_runners():
    dudes = {'Isaac': 2, 'Jack': 1, 'Josh': 3, 'Romo': 0}
    
    runners = []
    
    for name, base in dudes.items():
        runner = Baserunner(name, base)
        runner.set_master_status_unforced()
        
        runners.append(runner)
        
    return runners

In [27]:
def print_runners(runners):
    
    for runner in runners:
        print(f"\n{runner.name}: | ", end = "")
        
        for key, value in runner.master_status.items():
            print(f"{key}: {value}", end = " | ")

In [34]:
runners = make_runners()

print_runners(runners)


Isaac: | last_obtained: 2 | next_obtainable: 3 | preceding_obtainable: 1 | 
Jack: | last_obtained: 1 | next_obtainable: 2 | preceding_obtainable: 0 | 
Josh: | last_obtained: 0 | next_obtainable: 1 | preceding_obtainable: -1 | 
Romo: | last_obtained: 0 | next_obtainable: 1 | preceding_obtainable: -1 | 

## 3. Review all runners to determine who is forced

#### Define terms 

Runner R is forced to base "B + 1" if:
- R occupies B, and
- B-1 is occupied, and
- B-1 is forced

### Step 3.1 -- unpack bases occupied
- For this rules engine, I want to have a single source of truth (SSoT), and then unpack from that as needed.
- Here, the SSoT for which runners occupy which base is the runner objects.  

In [77]:
def unpack_runners_on(runners):

    bases = {0: True, 1: False, 2: False, 3: False}
    
    for runner in runners:
        
        R = runner.master_status['last_obtained']
        
        if R in [1, 2, 3]:
            bases[R] = runner.name
            
    return bases

### Step 3.2 Determine which bases are forced
- Go in order from 1B
- R1 is always forced
- R2 is forced to 3B if:
    - R2 occupies 2B, and
    - 1B is occupied, and
    - 1B is forced
- R3 is forced ... in a similar way

In [113]:
def print_tracer_1(bases, runners_on, bases_forced_to):
    
    for base in bases:
        print(f"Runner on {base}B: {runners_on[base]}", end = " | ")
        print(f"{base - 1}B is occupied: {runners_on[base - 1]}", end = " | ")
        print(f"{base - 1}B is forced to: {bases_forced_to[base - 1]}", end = " | ")
        print(f"So, {base + 1}B is forced: {bases_forced_to[base + 1]} ")

In [171]:
def set_forced_bases(runners_on):
    
    ## Initially define bases that a preceding runner is forced to
    
    bases_forced_to = {0: True, 1: True, 2: False, 3: False, 4: False}
    
    if runners_on[1]:
        bases_forced_to[2] = True

    #print_tracer_1( [1], runners_on, bases_forced_to)

        
    for base in [2, 3]:
        if runners_on[base]:
            if runners_on[base - 1]:
                if bases_forced_to[base - 1]:
                    bases_forced_to[base + 1] = True
                    
    #print_tracer_1( [2, 3], runners_on, bases_forced_to )
                        
    return bases_forced_to    

### Step 3.3 Assign an applicable forced base to each runner object 

In [169]:
def update_runner_bases(runners, bases_forced_to):
    
    for base in [2, 3, 4]: ## View as potential bases forced to
        
        if bases_forced_to[base]: ## If the potential is actually forced to
            
            for runner in runners: ## Find the runner occupying the preceding base and update their source of truth
                
                if runner.master_status['last_obtained'] == base - 1:

                    #print(f"\n{runner.name} forced to {base}\n")
                    
                    runner.set_master_status_forced()
    
    return runners

In [188]:
def set_forced(runners):
    
    #print_runners(runners) 
    #print("\n\n*******\n")

    runners_on = unpack_runners_on(runners)

    print("\n\n*******\n", "Runners on: ", runners_on, "\n\n*******\n")
    
    bases_forced_to = set_forced_bases(runners_on)

    
    runners = update_runner_bases(runners, bases_forced_to)
    
    #print("\nBases forced to: ", bases_forced_to)
    #print("\n*******")

    #print_runners(runners) 
    
    
runners = make_runners()

set_forced(runners)



*******
 Runners on:  {0: True, 1: 'Jack', 2: 'Isaac', 3: 'Josh'} 

*******



### Does it work if I remove the force?
- No, not as is... because I remove the 'last obtained' 

In [189]:
runners = make_runners()

for runner in runners:
    runner.print_status()
    print()

set_forced(runners)

print("\n\n")

for runner in runners:
    runner.print_status()
    print()

Isaac: 1B: preceding_obtainable  | 2B: last_obtained  | 3B: next_obtainable  | 4B: None  | 
Jack: 1B: last_obtained  | 2B: next_obtainable  | 3B: None  | 4B: None  | 
Josh: 1B: None  | 2B: preceding_obtainable  | 3B: last_obtained  | 4B: next_obtainable  | 
Romo: 1B: next_obtainable  | 2B: None  | 3B: None  | 4B: None  | 


*******
 Runners on:  {0: True, 1: 'Jack', 2: 'Isaac', 3: 'Josh'} 

*******




Isaac: 1B: None  | 2B: None  | 3B: forced_to  | 4B: None  | 
Jack: 1B: None  | 2B: forced_to  | 3B: None  | 4B: None  | 
Josh: 1B: None  | 2B: None  | 3B: None  | 4B: forced_to  | 
Romo: 1B: next_obtainable  | 2B: None  | 3B: None  | 4B: None  | 
